In [ ]:
import sqlite3

db = sqlite3.connect('..\cookbook.db')
cursor = db.cursor()

In [ ]:
# information
cursor.execute('''
CREATE TABLE IF NOT EXISTS information (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(255) NOT NULL,
    description TEXT,
    aggregated_rating DECIMAL(3, 2),
    recipe_servings INT,
    recipe_yield VARCHAR(255),
    recipe_instructions TEXT,
    cook_time INT,
    prep_time INT,
    total_time INT
);
''')

In [ ]:
# ingredients
cursor.execute('''
CREATE TABLE IF NOT EXISTS ingredients (
    ingredient_id INTEGER PRIMARY KEY AUTOINCREMENT,
    ingredient_name TEXT UNIQUE
);

''')

In [ ]:
# macros
cursor.execute('''
CREATE TABLE IF NOT EXISTS macros (
    recipe_id INT REFERENCES information(id) ON DELETE CASCADE,
    calories DECIMAL(5, 2),
    fat_content DECIMAL(5, 2),
    saturated_fat_content DECIMAL(5, 2),
    cholesterol_content DECIMAL(5, 2),
    sodium_content DECIMAL(5, 2),
    carbohydrate_content DECIMAL(5, 2),
    fiber_content DECIMAL(5, 2),
    sugar_content DECIMAL(5, 2),
    protein_content DECIMAL(5, 2),
    PRIMARY KEY (recipe_id)
);
''')

In [ ]:
# categories
cursor.execute('''
CREATE TABLE IF NOT EXISTS categories (
    category_id INTEGER PRIMARY KEY AUTOINCREMENT,
    category_name VARCHAR(255) UNIQUE
);
''')

In [ ]:
# recipe_categories
cursor.execute('''
CREATE TABLE IF NOT EXISTS recipe_categories (
    recipe_id INT REFERENCES information(id) ON DELETE CASCADE,
    category_id INT REFERENCES categories(category_id) ON DELETE CASCADE,
    PRIMARY KEY (recipe_id, category_id)
);
''')

In [ ]:
# keywords
cursor.execute('''
CREATE TABLE keywords (
    keyword_id INTEGER PRIMARY KEY AUTOINCREMENT,
    keyword_name VARCHAR(255) UNIQUE
);
''')

In [ ]:
# recipe_keywords
cursor.execute('''
CREATE TABLE recipe_keywords (
    recipe_id INT REFERENCES information(id) ON DELETE CASCADE,
    keyword_id INT REFERENCES keywords(keyword_id) ON DELETE CASCADE,
    PRIMARY KEY (recipe_id, keyword_id)
);
''')

In [ ]:
# generated descriptions
cursor.execute('''
CREATE TABLE generated_descriptions (
    recipe_id INT REFERENCES information(id),
    generated_description TEXT,
    PRIMARY KEY (recipe_id)
);
''')

In [6]:
import pandas as pd

df = pd.read_csv('../dataset/src.csv')

In [ ]:
information_df = df[['name', 'description', 'aggregated_rating', 'recipe_servings',
                  'recipe_yield', 'recipe_instructions', 'cook_time', 'prep_time', 'total_time']]
information_df.to_sql('information', db, if_exists='append', index=False)

In [ ]:
df.head()

In [12]:
# Find all of the unique ingredients
igr = set()
def parse_ingredients(s):
    ing = list(map(lambda x: x.strip('\'\" '), s[2:-1].split(',')))
    for i in ing:
        igr.add(i)

for line in df['recipe_ingredient_parts']:
    parse_ingredients(line)

In [ ]:
len(igr)

In [ ]:
ingredients_df = pd.DataFrame(igr, columns=['ingredient_name'])
ingredients_df.to_sql('ingredients', db, if_exists='append', index=False)

In [ ]:
unique_keywords = set()

def parse_list_of_items(s, set_to_add_to):
    if not isinstance(s, str):
        return
    if 'c' not in s:
        unique_keywords.add(s.strip('\'\" '))
        return
    lst = list(map(lambda x: x.strip('\'\" '), s[2:-1].split(',')))
    for i in lst:
        unique_keywords.add(i)
        
for r in df['keywords']:
    parse_list_of_items(r, unique_keywords)
unique_keywords

In [ ]:
keywords_df = pd.DataFrame(unique_keywords, columns=['keyword_name'])
keywords_df.to_sql('keywords', db, if_exists='append', index=False)

In [17]:
unique_categories = set([x for x in df['recipe_category'] if isinstance(x, str)])

In [ ]:
unique_categories
len(unique_categories)

In [ ]:
categories_df = pd.DataFrame(unique_categories, columns=['category_name'])
categories_df.to_sql('categories', db, if_exists='append', index=False)

In [ ]:
# macros processing
macros_df = df[['Unnamed: 0', 'calories', 'fat_content', 'saturated_fat_content', 'cholesterol_content',
                'sodium_content', 'carbohydrate_content', 'fiber_content', 'sugar_content', 'protein_content']]
macros_df = macros_df.rename(columns={'Unnamed: 0': 'recipe_id'}, inplace=False)
macros_df['recipe_id'] = macros_df['recipe_id'].apply(lambda x: x + 1)
macros_df.to_sql('macros', db, if_exists='append', index=False)

In [ ]:
# recipe_keywords processing
keywords_df = pd.read_sql('SELECT * FROM keywords', db)
keywords_mappping = dict(zip(keywords_df['keyword_name'], keywords_df['keyword_id']))

def parse_list_of_items(s):
    if not isinstance(s, str):
        return []
    if 'c' not in s:
        return [(s.strip('\'\" '))]
    lst = list(map(lambda x: x.strip('\'\" '), s[2:-1].split(',')))
    return lst

recipe_id_to_keywords = []

def generate_recipe_to_keyword(keywords_section, id):
    extracted_keywords = parse_list_of_items(keywords_section)
    recipe_id_to_keywords.append(extracted_keywords)
for i in range(len(df)):
    generate_recipe_to_keyword(df['keywords'][i], df['recipe_id'][i])
    
recipe_id_to_keywords

In [ ]:
keyword_list_to_keyword_ids = []
for keyword_list in recipe_id_to_keywords:
    keyword_ids = [keywords_mappping[keyword] for keyword in keyword_list]
    keyword_list_to_keyword_ids.append(keyword_ids)
keyword_list_to_keyword_ids

In [ ]:
print(len(keyword_list_to_keyword_ids))
average_keywords_per_recipe = sum([len(x) for x in keyword_list_to_keyword_ids]) / len(keyword_list_to_keyword_ids)
average_keywords_per_recipe

In [24]:
for i, keyword_ids in enumerate(keyword_list_to_keyword_ids):
    for keyword_id in keyword_ids:
        cursor.execute('INSERT INTO recipe_keywords (recipe_id, keyword_id) VALUES (?, ?)', (i + 1, keyword_id))

In [25]:
db.commit()

In [ ]:
# recipe_categories processing
categories_df = pd.read_sql('SELECT * FROM categories', db)
categories_mapping = dict(zip(categories_df['category_name'], categories_df['category_id']))
recipe_categories_df = df[['recipe_id', 'recipe_category']]
recipe_categories_df = recipe_categories_df.rename(columns={'recipe_category': 'category_id'})

def transform_category(category):
    return categories_mapping[category] if category in categories_mapping else None

recipe_categories_df['category_id'] = recipe_categories_df['category_id'].apply(transform_category)

recipe_categories_df.to_sql('recipe_categories', db, if_exists='append', index=False)

In [ ]:
# ingredient_recipe processing
cursor = db.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS ingredient_recipe (
    ingredient_id INT REFERENCES ingredients(ingredient_id) ON DELETE CASCADE,
    recipe_id INT REFERENCES information(id) ON DELETE CASCADE,
    quantity TEXT,
    PRIMARY KEY (recipe_id, ingredient_id)
);
''')

In [ ]:
# ingredient_recipe processing part 2
ingredients_df = pd.read_sql('SELECT * FROM ingredients', db)
ingredients_mapping = dict(zip(ingredients_df['ingredient_name'], ingredients_df['ingredient_id']))
recipe_id_to_ingredients_to_quantity = []

def parse_quantity(s):
    if not isinstance(s, str):
        return []
    if 'c' not in s:
        return [(s.strip('\'\" '))]
    lst = list(map(lambda x: x.strip('\'\" '), s[2:-1].split(',')))
    return lst

def parse_ingredients(s):
    ing = list(map(lambda x: x.strip('\'\" '), s[2:-1].split(',')))
    return ing

def generate_recipe_to_ingredient(ingredients_section, quantities_section, id):
    extracted_ingredients = parse_ingredients(ingredients_section)
    extracted_quantities = parse_quantity(quantities_section)
    recipe_id_to_ingredients_to_quantity.append((extracted_ingredients, extracted_quantities))
    
for i in range(len(df)):
    generate_recipe_to_ingredient(
        df['recipe_ingredient_parts'][i],
        df['recipe_ingredient_quantities'][i],
        df['recipe_id'][i]
    )

recipe_id_to_ingredients_to_quantity[1]

In [ ]:
recipe_id_to_ingredients_to_quantity

In [29]:
cursor = db.cursor()

recipe_id_to_ingredient_id_to_quantity = []
for i, (ingredients, quantities) in enumerate(recipe_id_to_ingredients_to_quantity):
    ingredient_id_to_quantity = []
    for ingredient, quantity in zip(ingredients, quantities):
        ingredient_id = ingredients_mapping[ingredient]
        ingredient_id_to_quantity.append((ingredient_id, quantity))
    recipe_id_to_ingredient_id_to_quantity.append(ingredient_id_to_quantity)

In [31]:
cursor = db.cursor()

for i, row in enumerate(recipe_id_to_ingredient_id_to_quantity):
    for ing_id, qtty in row:
        cursor.execute('SELECT COUNT(*) FROM ingredient_recipe WHERE ingredient_id = ? AND recipe_id = ?', (ing_id, i + 1))
        if cursor.fetchone()[0] == 0:
            cursor.execute('INSERT INTO ingredient_recipe (ingredient_id, recipe_id, quantity) VALUES (?, ?, ?)', (ing_id, i + 1, qtty))


In [8]:
gen_desc_df = pd.read_csv('../dataset/src_with_desc.csv')

In [10]:
for i in range(len(gen_desc_df)):
    cursor.execute('INSERT INTO generated_descriptions (recipe_id, generated_description) VALUES (?, ?)', (i + 1, gen_desc_df['generated_description'][i]))

In [11]:
db.commit()

In [33]:
db.close()